In [9]:
#for python 2/3 compatability
#from __future__ import print_function

#sample dataset
# format: each row is an example
# The last column is the label
# The first two columns are features
# If you want you can add more features and examples
# Interesting note: 2nd and 5th examples have the same features, but different labels
# Let's see how tree handles this case.

training_data= [
    ['Green',3,'Mango'],['Yellow',3,'Mango'],['Red',1,'Grape'],
    ['Red',1,'Grape'],['Yellow',3,'Lemon']
]

# Column labels
# These are used only to print the tree

header=["color","diameter","label"]

def unique_vals(rows,col):
  """Find the unique values for a column in a dataset."""
  return set(row[col]for row in rows)

# Demo:
# unique_vals(training_data,0)
# unique_vals(training_data,1)

def class_counts(rows):
  """Counts the number of each type of example in a dataset."""
  counts= {} # a dictionary of label -> count.
  for row in rows:
    # in our dataset format, the label is always the lost column.
    label= row[-1]
    if label not in counts:
      counts[label]= 0
    counts[label] +=1
  return counts

# Demo:
# class_counts(training_data)

def is_numeric(value):
  """Test if a value is numeric"""
  return isinstance(value, int) or isinstance(value, float)

# Demo:
# is_numeric(7)
# is_numeric("Red")

class Question:
  """A question is used to partition a dataset.
  This class just records a 'column number' (e.g., 0 for Color) and a
  'column value' (e.g., Green). The 'match' method is used to compare
  the feature value in an example to the feature value stored in the
  question. see the demo:"""

  def __init__(self, column, value):
    self.column= column
    self.value= value

  def match(self, example):
    # Compare the feature value in an example to the feature value in this question
    val= example[self.column]
    if is_numeric(val):
      return val>=self.value
    else:
      return val == self.value

  def __repr__(self):
    # This is just a helper method to print the question in a readable format
    condition= "=="
    if is_numeric(self.value):
      condition= ">="
    return "Is %s %s %s?" % (
        header[self.column], condition, str(self.value)
    )

def partition(rows, question):
  """Partitions the dataset.
  For each row in the dataset, check if it matches the question. If so,
  and it to 'true rows', otherwise, add it to 'false rows'."""
  true_rows, false_rows= [], []
  for row in rows:
    if question.match(row):
      true_rows.append(row)
    else:
      false_rows.append(row)
  return true_rows, false_rows

# Demo:
# Let's partition the training data based on whether rows are Red.
# true_rows, false_rows= partition(training_data, Question(0, 'Red'))
# This will contain all Red rows.
# true_rows
# This will contain everything else.
# false_rows

def gini(rows):
  """Calculate the gini impurity for a list of rows."""
  counts= class_counts(rows)
  impurity=1
  for lbl in counts:
    prob_of_lbl= counts[lbl]/float(len(rows))
    impurity -= prob_of_lbl**2
  return impurity

# Demo:
# Let's look at some example to understand how gini impurity works.
# First we'll look at a dataset with no mixing.
# no_mixing =[['Mango'],['Mango']]
# This will return 0
# Lots_of_mixing= [['Mango'],['Orange'],['Grape'],['Bleberry']]
# This will return 0.8
# gini(Lots_of_mixing)

def info_gain(left, right, current_uncertainty):
  """Information gain.
  The uncertainty of the starting mode, minus the weighted impurity of the child nodes"""
  p= float(len(left)/(len(left)+len(right)))
  return current_uncertainty-p*gini(left)-(1-p)*gini(right)

def find_best_split(rows):
  """Find the best question to ask by iterating over every feature / value
  and calculating the info gain."""
  best_gain= 0 # keep track of the best info gain
  best_question= None # keep train of the feature / value that producd it
  current_uncertainty= gini(rows)
  n_features= len(rows[0])-1 # number of columns

  for col in range(n_features): # for each feature
    values= set([row[col] for row in rows]) # unique values in the column
    for val in values: # for each value
      question= Question(col, val)
    # try splitting the dataset
      true_rows, false_rows= partition(rows, question)
    # skip this split if it doesn't divide the dataset.
      if len(true_rows) == 0 or len(false_rows) == 0:
        continue
    # Calculate the info gain from this split
      gain= info_gain(true_rows, false_rows, current_uncertainty)

    # You actually can use '>' instead of '>=' here but i wanted the tree to look a certain way for our dataset.
      if gain >= best_gain:
        best_gain, best_question= gain, question
  return best_gain, best_question

class Leaf:
  def __init__(self,rows):
    self.predictions= class_counts(rows)

class Decision_node:
  def __init__(self, question, true_branch, false_branch):
    self.question= question
    self.true_branch= true_branch
    self.false_branch= false_branch

def build_tree(rows):
  gain, question= find_best_split(rows)
  if gain == 0:
    return Leaf(rows)
  true_rows, false_rows= partition(rows, question)
  true_branch= build_tree(true_rows)
  false_branch= build_tree(false_rows)

  return Decision_node(question, true_branch, false_branch)

def print_tree(node, spacing=""):
  if isinstance(node, Leaf):
    print(spacing + "Predict", node.predictions)
    return
  print(spacing + str(node.question))
  print(spacing + '--> True:')
  print_tree(node.true_branch, spacing + " ")
  print(spacing + '--> False:')
  print_tree(node.false_branch, spacing + " ")

def classify(row, node):
  if isinstance(node, Leaf):
    return node.predictions

  if node.question.match(row):
    return classify(row, node.true_branch)
  else:
    return classify(row, node.false_branch)

def print_leaf(counts):
  total= sum(counts.values())*1.0
  probs={}
  for lbl in counts.keys():
    probs[lbl]= str(int(counts[lbl]/total*100))+ "%"
  return probs

if __name__=='__main__':
  my_tree = build_tree(training_data)
  print_tree(my_tree)

  testing_data=[['Green',3,'Mango'],['Yellow',4,'Mango'],['Red',2,'Grape'],
    ['Red',1,'Grape'],['Yellow',3,'Lemon']]

  for row in testing_data:
    print("Actual: %s. Predicted: %s." %
          (row[-1], print_leaf(classify(row, my_tree))))

Is diameter >= 3?
--> True:
 Is color == Yellow?
 --> True:
  Predict {'Mango': 1, 'Lemon': 1}
 --> False:
  Predict {'Mango': 1}
--> False:
 Predict {'Grape': 2}
Actual: Mango. Predicted: {'Mango': '100%'}.
Actual: Mango. Predicted: {'Mango': '50%', 'Lemon': '50%'}.
Actual: Grape. Predicted: {'Grape': '100%'}.
Actual: Grape. Predicted: {'Grape': '100%'}.
Actual: Lemon. Predicted: {'Mango': '50%', 'Lemon': '50%'}.
